## Location visualisation template

In [ ]:
# Import libraries
import pandas as pd
import json

#in order of complexity we import the different plotly libraries
import plotly.express as px
import plotly.graph_objects as go
import datashader as ds

import dask.dataframe as dd #alternative to pandas, is faster with large datasets

In [ ]:
# Load some data with longitude and langitude
file_path = './data/df_2020.csv'
df_pd = pd.read_csv(file_path, low_memory=False)
df_dd = dd.read_csv(file_path)

In [ ]:
# with plotly.express it is very easy to create a scatterplot with a map, here an example with data from the library
df_iso = px.data.gapminder().query("year == 2007")
fig = px.scatter_geo(df_iso, locations="iso_alpha", # in this case the locations are based on iso codes, e.g. the Netherlands is NLD
                     size="pop", # size of markers, "pop" is one of the columns of gapminder
                     )
fig.show()

In [ ]:
fig_map_box = px.scatter_mapbox(df_pd, #which data to use, just a pandas dataframe
                        lat='latitude', #which column to use for latitude
                        lon='longitude', #which column to use for longitude
                        hover_data =['accident_index'], #what data to show when hovering over a data point
                        color='accident_severity', # what colour to give the data, can work with both numerical and categorical variables
                        size_max=100,
                        mapbox_style='carto-darkmatter')

fig_map_box.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0},
                          mapbox_zoom=4, # use this to zoom in on the map
                          mapbox_center_lat = 54.5, # use this to align the map on latitude
                          mapbox_center_lon = -2 # use this to align the map on longitude
                         )
fig_map_box.show()

In [ ]:

fig_den_box = px.density_mapbox(df_pd, #which data to use, just a pandas dataframe
                        lat='latitude', #which column to use for latitude
                        lon='longitude', #which column to use for longitude
                        hover_data =['accident_index'], #what data to show when hovering over a data point
                        z='accident_severity', # what colour to give the data, can work with both numerical and categorical variables
                        mapbox_style='carto-darkmatter',
                        opacity = 0.5, # change the opacity of the oclours on the map
                        range_color = [0, 100]) # change the range of the z metric's colour scale)

fig_den_box.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0},
                        mapbox_zoom=4, # use this to zoom in on the map
                        mapbox_center_lat = 54.5, # use this to align the map on latitude
                        mapbox_center_lon = -2 # use this to align the map on longitude
                         )
fig_den_box.show()

In [ ]:
# getting the geojson data (boandary data) for the UK
with open('Data/uk_local_areas/uk_la.geojson') as f:
    geojson = json.load(f)

# plot a choropleth plot with the json data
fig = px.choropleth(df_pd, geojson=geojson, color="accident_severity",
                    locations="local_authority_ons_district", featureidkey="properties.geo_code",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# make a fig object with a plot attached with the pandas df (WARNING: very slow)
fig = go.Figure(data=go.Scattergeo(
        lon = df['longitude'],
        lat = df['latitude'],
        text = df['accident_index'],
        mode = 'markers',
        marker_color = df['accident_severity'],
        ))

fig.update_layout(
        title = 'Accidents<br>Great Britain 2020',
        geo_scope='europe',
    )

fig.show()

In [ ]:
#creating a datashader canvas
cvs = ds.Canvas(plot_width=1000, plot_height=1000)
agg = cvs.points(df, x='longitude', y='latitude')
# agg is an xarray object, see http://xarray.pydata.org/en/stable/
coords_lat, coords_lon = agg.coords['latitude'].values, agg.coords['longitude'].values
# Corners of the image, which need to be passed to mapbox
coordinates = [[coords_lon[0], coords_lat[0]],
               [coords_lon[-1], coords_lat[0]],
               [coords_lon[-1], coords_lat[-1]],
               [coords_lon[0], coords_lat[-1]]]